This cell installs code that helps us run the rest of the notebook.

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null
!git clone https://github.com/edgi-govdata-archiving/EEW_Industry_Reports.git &>/dev/null
import pandas
import folium
import ipywidgets as widgets
import pdb
import os
import urllib.parse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

This cell allows the user to make selections regarding programs, active facilities, and NAICS codes. make_selections() displays a widget, allowing the user to choose multiple programs from a list, decide if they want to filter by active facilities, and input specific NAICS codes separated by commas. Returns the selections made in the widget.

In [ ]:
from EEW_Industry_Reports.backend_scripts.programs_facilities_codes import make_selections
slct = make_selections()

This cell generates an SQL query based on the selections made by the user in the previous cell. query(slct) takes in the arg slct which contains the selections from the previous cell. Returns the SQL query string.

In [ ]:
from EEW_Industry_Reports.backend_scripts.query_and_filter import query
squery = query(slct)

# Access the ECHO_EXPORTER table
# Output will be the matching tracked by EPA
from ECHO_modules.ECHO_modules.get_data import get_echo_data

sql = 'select "FAC_NAME", "CWA_NAICS", "CAA_NAICS", "RCRA_NAICS", "FAC_NAICS_CODES", "FAC_LAT", "FAC_LONG", "FAC_INSPECTION_COUNT", "FAC_DAYS_LAST_INSPECTION" ,"FAC_INFORMAL_COUNT" ,"FAC_FORMAL_ACTION_COUNT", "FAC_PERCENT_MINORITY", "FAC_TOTAL_PENALTIES", "FAC_PENALTY_COUNT", "FAC_QTRS_WITH_NC","EJSCREEN_FLAG_US" from "ECHO_EXPORTER" where ' + squery
try:
  echo = get_echo_data( sql )
except pandas.errors.EmptyDataError:
  print("\nThere are no records.\n")

echo

This cell outputs the mean and medians of different selected fields from the echo table such as FAC_PERCENT_MINORITY, FAC_TOTAL_PENALTIES, etc.

In [ ]:
from EEW_Industry_Reports.backend_scripts.mean_median import metrics
metrics(echo)

This cell generates box plots of different fields.

In [ ]:
from EEW_Industry_Reports.backend_scripts.graphics import plot_visual_data_box_plot
plot_visual_data_box_plot(echo.FAC_QTRS_WITH_NC, "FAC_QTRS_WITH_NC", "x", "y")
plot_visual_data_box_plot(echo.FAC_PERCENT_MINORITY, "FAC_PERCENT_MINORITY", "x", "y" )
plot_visual_data_box_plot(echo.FAC_TOTAL_PENALTIES, "FAC_TOTAL_PENALTIES", "x", "y" )
plot_visual_data_box_plot(echo.FAC_PENALTY_COUNT, "FAC_PENALTY_COUNT", "x", "y" )

In [ ]:
# Export the data to the "Files" side panel for download
echo.to_csv("data.csv")

Mapping the facilities, displaying the name and inspection count of each.

In [ ]:
# Map the facilities
from folium.plugins import MarkerCluster
import math

map = folium.Map()
marker_cluster = MarkerCluster().add_to(map)

for index, row in echo.iterrows():
  if math.isnan(row["FAC_LAT"]) or math.isnan(row["FAC_LONG"]): #ignore facilities with no spatial data
    pass
  else:
    marker = folium.Marker(
        location = [float(row["FAC_LAT"]), float(row["FAC_LONG"])],
        popup = row["FAC_NAME"] + ", Inspection count: " + str(row["FAC_INSPECTION_COUNT"])
    ).add_to(marker_cluster)

map
